<img src=https://github.com/computationaljournalism/columbia2018/raw/master/images/maxresdefault.jpg width=500>
    

**Collaborative filtering**

Now we are going to lay the groundwork for a cluster of computational tools that are riled under the names "statistical modeling", "machine learning", and even "Artifical Intelligence". We've seen various learning systems in previous lectures, and today we're going to examine so-called **recommender system**. In the material above, we talked about the basics of a **content-based** recommendation system. In the case of news, for example, recommended articles could be surfaced if you express interest in one topic or another. Other information about the stories could also be used to produce a ranking -- you might also include the number of "likes" or tweets, the publisher, the publication date and so on. 

This is the first of three approaches Alex Spangher wrote about in his NYT piece on their [next generation news recommender](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/). The second approach can be implemented without any information about content, and instead relies on how users rate or access content. On Amazon or Yelp or TripAdvisor, we have ratings that people provide for services. At the Times, we don't really have ratings, but instead we have whether or not someone read an article. **Collaborative filtering** uses patterns in these activities to make recommendations. (We "filter" items, recommending just a few, and this is a "collaborative" act because it relies on the input of others.)

At the heart of collaborative filtering is something called a "utility matrix" or "incidence matrix". It's a table where rows refer to users (shoppers, readers), say, and columns are items (news articles, consumer goods). A recommendation system "fills in" or predicts how the user in row i will react to the object represented by column j -- that is, the table entry (i,j). Oh, math!

So, here's a cartoon. Let's decide if we are going to recommend a great article on Climate Change to Marie. Marie is represented by row i in the incidence matrix and our Climate Change article is represented by column j. In row i we have 0's and 1's -- a 0 if Marie has not read the article and a 1 if she has. Column j is also filled with 0's and 1's, but this time an entry is 0 if the corresponding reader hasn't selected the article and 1 if he has. Got it? A row refers to the choices by a single user, and a column refers to the choices made by all our users for a single article. 

Now, we have two possible ways to use our incidence matrix, our table of 0's and 1's, to predict whether Marie will like our Climate Change article. The first is to find readers that are "similar" to Marie in terms of reading habits. That is, they have read some of the same articles or have 0's and 1's in the same places across their rows. We can take some number of readers who are most similar to Marie (the number always being called k) and then see what fraction read the article in question. In the image below, the gray band refers to the k readers who are similar to Marie and the red stripe highlights a column representing their reading of the single article. 

<img src=https://github.com/computationaljournalism/columbia2018/raw/master/images/abcabc.001.jpeg width=700>

The second approach to fill in the (i,j) entry of a table like this would be to look at  articles similar to the Climate Change piece we want to recommend to Marie. Articles are similar if they were read (or not) by the same people. We can then take the k most similar articles and see what fraction of them Marie has read. If it's high, we recommend the article to her. The figure below is just the flip of the one we've seen. The grey band is now similar articles to the given one and the red stripe looks at whether or not Marie read these. 

<img src=https://github.com/computationaljournalism/columbia2018/raw/master/images/abcabc.002.jpeg width=700>

This seems pretty straightforward (I hope). The techniques here are referred to ask "k-nearest neighbors" (or KNN or kNN). It's actually a pretty powerful machine learning (well, statistical) technique. And we leverage this kind of scheme all the time -- your doctor, for example, distills "you" into a row in a table, a height, a weight, a blood pressure, maybe some facts from your medical history. She then gives you advice about dropping a few pounds, say by what the medical profession knows about people like you. The idea is that the health outcomes of people "similar" to you can help predict what's around the corner for you. 

The key ideas here are pretty fundamental and **they have to do with distance.** Rows, for example, can be compared. In the case of collaborative filtering, we can evaluate one reader relative to another, marking some as close (likeminded, similar tastes) and others as far (red state, different tastes). This is a basic, mathematical abstraction that machine learning applies to all tables. Rows are points in a "high-dimensional space". (The same is true for columns, of course, except that it's typical to have tables -- spreadsheets-- organized so that rows refer to units of observation and columns refer to their characteristics.)

Let's see an incidence matrix from a real application. This is a little perverse, but it was the best I could do, try as I might.

<img src=https://github.com/computationaljournalism/columbia2018/raw/master/images/recommender-systems-13-638.jpg width=500>
    

**Collaborative baking**

So far, this is pretty straightforward (I hope). The techniques here are referred to ask "k-nearest neighbors" (or KNN or kNN). It's actually a pretty powerful machine learning (well, statistical) technique. And we leverage this kind of scheme all the time -- your doctor, for example, distills "you" into a row in a table, a height, a weight, a blood pressure, maybe some facts from your medical history. She then gives you advice about dropping a few pounds, say by what the medical profession knows about people like you. The idea is that the health outcomes of people "similar" to you can help predict what's around the corner for you. 

The key ideas here are pretty fundamental and **they have to do with distance.** Rows, for example, can be compared. In the case of collaborative filtering, we can evaluate one recipe relative to another, marking some as close (similar ingredients) and others as far (different tastes). This is a basic, mathematical abstraction that machine learning applies to all tables. Rows are points in a "high-dimensional space". (The same is true for columns, of course, except that it's typical to have tables -- spreadsheets-- organized so that rows refer to units of observation and columns refer to their characteristics.)

Let's see an incidence matrix from our cakes data set. It is weenie, but I'm down the rabbit hole now.

In [ ]:
from pandas import read_csv
cakes = read_csv("https://github.com/computationaljournalism/columbia2019/raw/master/data/cakes.csv")

In [ ]:
cakes.head(5)

The first five rows of the `incidence` data frame. Again, the matrix holds a 0 in row i and column j if the recipe in row i is missing ingredient j. It's 1 otherwise. Remember how many cakes we had...

In [ ]:
cakes.shape

So, 1,000 recipes and 381 ingredients. As we commented in the last notebook, row and column sums can be important. Summing down the columns counts how many recipes (out of 1,000) contain each ingredient. Here we `apply()` the `sum()` function down columns, indicated with `axis=0`. We sort the resuts and see that many ingredients appear just once.

In [ ]:
cakes.apply(sum,axis=0).sort_values()

Here we compute just how many ingredients appear in one recipe. We take the column sum and ask for a boolean outcome, `True` if there was one recipe and `False` for more. Adding these up turns `True` to 1 and `False` to zero. 

In [ ]:
sum( cakes.apply(sum,axis=0) == 1)

That means 126 or about 13% are used by just one recipe. How many have two? Three? Less than five?

In [ ]:
# your code here



How should we judge similarity between two recipes? Again, we want them to have use much the same ingredients. A simple measure for that is the **Jaccard metric.** It is good for rows (or columns) that are made up of 0's and 1's. Essentially it looks at how many 1's the two recipes have in common, divided by the total number of ingredients they require. Well, you subtract that ratio from 1. So, if two recipes have nothing in common, the metric will be 1, and if they have everything in common, it will be 0.

In [ ]:
# Jaccard distance

def dist(a,b):
    
    intersection = sum((a+b)==2)
    union = sum((a+b)>=1)
    
    # print intersection,union
    
    return 1.0-(0.0+intersection)/union

Let's try out the function. We'll make two series (Pandas' representation of a row or column) of 0's and 1's and computes their Jaccard distance. See if this makes sense. Change the 0's and 1's to all agree or disagree.

In [ ]:
from pandas import Series
x = Series([1,0,1,1])
y = Series([0,0,1,1])
dist(x,y)

Finally, to compute the distance between two recipes, we need to access rows. This is done using `.iloc[]` subsetting. We haven't seen this yet, but it gives us fine-grained control over the rows or columns we want to extract. The notation is 
>`row selection, column selection`

Before the comma refers to rows and after to columns. We can use single integers for a single row or column, a slice like `5:10` to get a range,  and the empty slice `:` to ask for all the rows or columns. The result is a Pandas Series. 

Here is recipe with ID 40. 

In [ ]:
cakes.iloc[40,:]

This recipe has read 10 ingredients.

In [ ]:
sum(cakes.iloc[40,:])

Here are the 10.

In [ ]:
[c for c in cakes.columns if cakes[c][10]]

And here is the distance between recipe 40 and recipe 10.

In [ ]:
[c for c in cakes.columns if cakes[c][40]]

The two recipes share 8 ingredients and between them, there are 12 total ingredients. That means the distance is 1-8/12 = 1-2/3 = 1/3. Let's see!

In [ ]:
dist(cakes.iloc[40,:],cakes.iloc[10,:])

Yes! 

Now, this loop iterates through the entire set of rows and calculates the distance between recipe 40 and the rest. 

In [ ]:
distances = Series([ dist(cakes.iloc[40,:],cakes.iloc[i,:]) for i in range(1000)])
distances.head()

Here we sort the distaances and look at the distance of the 25th nearest recipe to number 40.

In [ ]:
close = distances.sort_values().reset_index(drop=True)[25]
close

We then subset just the nearby cakes, those with distance less than 0.417. 

In [ ]:
near_cakes = cakes[(distances<=close) & (distances>0)]
near_cakes.head(5)

Let's pull out the ingredients in recipe 40, storing them in a list.

In [ ]:
ingredients = [ing for ing in cakes.columns if cakes[ing][40]]
ingredients

Finally, we sort the ingredients according to the number of recipes that contain them. We then leave out all the ingredients that are already in cake 40. 

In [ ]:
recommendations = near_cakes.apply(sum,axis=0).sort_values(ascending=False).reset_index()
recommendations[~recommendations["index"].isin(ingredients)].head(10)

Let's wrap these two into a function and have a look at a few cakes and what we recommend to add.

In [ ]:
def dist(a,b):
    
    intersection = sum((a+b)==2)
    union = sum((a+b)>=1)
    
    # print intersection,union
    
    return 1.0-(0.0+intersection)/union

def recommend(c,k=10,recipes=cakes):
    
    n_recipes = recipes.shape[0]
    
    ingredients = [ing for ing in cakes.columns if cakes[ing][c]]
    print("Ingredients in recipe",c)
    print(ingredients)
    
    distances = Series([dist(recipes.iloc[c,:],recipes.iloc[i,:]) for i in range(n_recipes)])
    close = list(distances.sort_values())[k]

    near_recipes = recipes[(distances<=close) & (distances>0)]
    recommendations = near_recipes.apply(sum,axis=0).sort_values(ascending=False).reset_index()
    
    return recommendations[~recommendations["index"].isin(ingredients)][:10]

In [ ]:
recommend(920,25)

In [ ]:
recommend(230,25)

In [ ]:
recommend(430,25)